<a href="https://www.kaggle.com/code/mh0386/facial-emotions-detection?scriptVersionId=156333364" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
#----------------------------------------#
!pip install xgboost                     #
!pip install --upgrade                   #
#----------------------------------------#

In [ ]:
import os
import tensorflow as tf
print("Num GPUs Available:", (tf.config.experimental.list_physical_devices('TPU')))
from keras.applications.xception import Xception
import bz2
import os
import xgboost as xgb
import requests
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import img_to_array , load_img
from keras.models import load_model
import scipy
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as p_input
from keras.utils import to_categorical
import numpy as np
import cv2
import requests
from keras.utils import plot_model
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.datasets import make_classification
from deap import base, creator, tools, algorithms

In [ ]:
# #!pip install requests
# # import requests
# # import bz2
# import dlib

# # # Download the shape predictor file compressed in bz2 format
# # url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
# # response = requests.get(url)
# # with open("shape_predictor_68_face_landmarks.dat.bz2", "wb") as f:
# #     f.write(response.content)

# # # Decompress the downloaded bz2 file
# # with open("shape_predictor_68_face_landmarks.dat.bz2", "rb") as f:
# #     compressed_data = f.read()
# #
# # decompressed_data = bz2.decompress(compressed_data)
# #
# # # Save the decompressed data to a file
# # with open("shape_predictor_68_face_landmarks.dat", "wb") as f:
# #     f.write(decompressed_data)
# #
# # print("Shape predictor file downloaded and decompressed.")

# # Now you can use the dlib detector and predictor
# datFile = "shape_predictor_68_face_landmarks.dat"
# detector = dlib.get_frontal_face_detector()
# predictor = dlib.shape_predictor(datFile)


In [ ]:

#reading and Augmenting data and preprocess it for VGG16
# def custom_preprocess_input(img):
#     processed_img = p_input(img)
#     return processed_img

In [ ]:
# Load images and labels
general_path = "/kaggle/input/fer2013/"
path_train_dataset = general_path + "train/"
path_validation_dataset = general_path + "test/"
picture_size=224 #input of VGG16 should be 224 AND input for mibile network should be 227
batch_size=256
number_of_classes=7
# Data Augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    preprocessing_function=p_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Load train and test datasets
train_data_set = datagen.flow_from_directory(
    directory=path_train_dataset,
    target_size=(picture_size, picture_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_data_set = datagen.flow_from_directory(
    directory=path_validation_dataset,
    target_size=(picture_size, picture_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

In [ ]:
len(train_data_set.filenames)

In [ ]:
expression = "happy"
plt.figure(figsize=(12, 12))

for i in range(1, 13):
    plt.subplot(4, 4, i)
    img_path = os.path.join(path_train_dataset, expression,
                            os.listdir(os.path.join(path_train_dataset, expression))[i])
    img = load_img(img_path, target_size=(48, 48))
    plt.imshow(img)
    plt.title(f"Image {i}")
    plt.axis("off")
    plt.title(expression)
plt.show()

labels = []
images=[]




In [ ]:
expression = "angry"
plt.figure(figsize=(10, 10))

for i in range(1, 10):
    plt.subplot(3, 3, i)
    img_path = os.path.join(path_train_dataset, expression,
                            os.listdir(os.path.join(path_train_dataset, expression))[i])
    img = load_img(img_path, target_size=(48, 48))
    plt.imshow(img)
    plt.title(f"Image {i}")
    plt.axis("off")
    plt.title(expression)
plt.show()


In [ ]:
train_data_set.next()[0].shape #get shape of each patch 

# Now we will try to use exception Feature extraction architicture 

In [ ]:
from keras.utils import plot_model
non_trainable_model = Xception(weights='imagenet', include_top=False, input_shape=(picture_size, picture_size, 3))
for layer in non_trainable_model.layers:
    layer.trainable = False
# output_features_of_freezed_model = non_trainable_model.predict(train_data_set)
non_trainable_model.summary()
plot_model(non_trainable_model, to_file='Xception_base_model.png', show_shapes=True)

**Know we want to access extracted features of data to feed into XG_boost with genatic algorithm optimizer**

In [ ]:
train_data=non_trainable_model.predict(train_data_set)
test_data = train_data=non_trainable_model.predict(test_data_set)


In [ ]:
train_data.shape 

In [ ]:


# Assuming you have a dataset X and corresponding labels y
# For illustration purposes, I'm using a synthetic dataset

# X, y = make_classification(n_samples=1000, n_features=20, n_classes=3, random_state=42)


# Define the XGBoost fitness function for multi-class classification

# we use compination of f1_scores_ and recall to give high penality into false negative 
def xgboost_fitness(hyperparameters):
    params = {
        'objective': 'multi:softmax',
        'num_class': len(np.unique(y)),  # Number of classes
        'eval_metric': 'mlogloss',
        'eta': hyperparameters[0],  # Learning rate
        'max_depth': int(hyperparameters[1]),  # Maximum depth of a tree
        'subsample': hyperparameters[2],  # Subsample ratio of the training instances
        'colsample_bytree': hyperparameters[3],  # Subsample ratio of columns when constructing each tree
        'min_child_weight': int(hyperparameters[4]),  # Minimum sum of instance weight (hessian) needed in a child
    }

    # Train XGBoost model
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate metrics
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    # Combine metrics into a single fitness score
    fitness = f1 * (1 + 0.1 * (1 - recall))

    return fitness,

# Define genetic algorithm parameters
NUM_GENERATIONS = 10
POPULATION_SIZE = 10
CXPB, MUTPB = 0.7, 0.2

# Define the hyperparameter space
HYPERPARAMETER_SPACE = [
    ('eta', 0.01, 1.0),  # Learning rate
    ('max_depth', 1, 10),  # Maximum depth of a tree
    ('subsample', 0.1, 1.0),  # Subsample ratio of the training instances
    ('colsample_bytree', 0.1, 1.0),  # Subsample ratio of columns when constructing each tree
    ('min_child_weight', 1, 10),  # Minimum sum of instance weight (hessian) needed in a child
]

# Create a fitness function for the genetic algorithm
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)




#--------------------------------------------------------------------------------------------------------------------
#this is the steps of genatic algorithm 
#first initialize individual at the start of first cycle of generation 
#second initialize the population of last initialized  individuales
#check the fitness value
#check make crossover and mutation  using gaussian distibutionwith CXPB, MUTPB probabilities of crossover and mutation 
# select the high fitness value and make a new generation cycle and so on until you reach the optimal hyperparams 
#---------------------------------------------------------------------------------------------------------------------
#toolbox is a container for using genatic algorithm 
#---------------------------------------------------------------------------------------------------------------------



toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, low=0, high=1)
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_float, ), n=len(HYPERPARAMETER_SPACE))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", xgboost_fitness)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Main genetic algorithm loop
population = toolbox.population(n=POPULATION_SIZE)

for gen in range(NUM_GENERATIONS):
    offspring = algorithms.varAnd(population, toolbox, cxpb=CXPB, mutpb=MUTPB)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for ind, fit in zip(offspring, fits):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))

# Get the best individual (best set of hyperparameters)
best_individual = tools.selBest(population, k=1)[0]
best_hyperparameters = [param[1].low + value * (param[1].high - param[1].low) for param, value in zip(HYPERPARAMETER_SPACE, best_individual)]

print("Best Hyperparameters:", best_hyperparameters)



VGG16 Model

In [ ]:
# Model=tf.keras.Sequential(
# [
#         non_trainable_model, #the extracted features from VGG16 will be used as input of dense layers which is new model ("we applied transfer learning ")
#         tf.keras.layers.Flatten(),
#         tf.keras.layers.Dense(512, activation=tf.nn.relu),
#         tf.keras.layers.Dense(256, activation=tf.nn.relu),
#         tf.keras.layers.Dense(128, activation=tf.nn.relu),
#         tf.keras.layers.Dense(number_of_classes, activation=tf.nn.softmax)
#     ]
# )
# Model.summary()
# from keras.utils import plot_model
# Model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])



In [ ]:
# Model.fit_generator(
#     train_data_set,
#     steps_per_epoch=train_data_set.n // train_data_set.batch_size,
#     epochs=10,
#     validation_data=test_data_set,
#     validation_steps=test_data_set.n // test_data_set.batch_size
    
# )

In [ ]:
# # plot_model(Model, to_file='model_architecture.png', show_shapes=True)
# # Save the model
# Model.save('face_emotion_model_VGG16.h5')



In [ ]:
# # Plot the accuracy and loss curves
# fig, axes = plt.subplots(1, 2, figsize=(20, 5))
# axes[0].plot(history.history['accuracy'], label='Train accuracy')
# axes[0].plot(history.history['val_accuracy'], label='Validation accuracy')
# axes[0].set_xlabel('Epochs')
# axes[0].legend()
# axes[1].plot(history.history['loss'], label='Train loss')
# axes[1].plot(history.history['val_loss'], label='Validation loss')
# axes[1].set_xlabel('Epochs')
# axes[1].legend()
# plt.show()

In [ ]:
# import cv2
# import dlib
# import numpy as np
# from keras.models import load_model
# from keras.preprocessing.image import img_to_array


# model=load_model(".h5")
# # Load face detector from dlib
# detector = dlib.get_frontal_face_detector()

# # Define the emotion labels
# emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# # Initialize the webcam
# cap = cv2.VideoCapture(0)

# while True:
#     # Read a frame from the webcam
#     ret, frame = cap.read()

#     if not ret:
#         break

#     # Convert the frame to grayscale
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Detect faces in the frame
#     faces = detector(gray)

#     for face in faces:
#         x, y, w, h = face.left(), face.top(), face.width(), face.height()

#         # Extract the face region
#         face_roi = gray[y:y+h, x:x+w]
#         face_roi = cv2.resize(face_roi, (224, 224))
#         face_roi = p_input(face_roi)
#         face_roi = img_to_array(face_roi)
#         face_roi = np.expand_dims(face_roi, axis=0)

#         # Predict the emotion using the model
#         emotion_probabilities = model.predict(face_roi)[0]
#         predicted_emotion = emotion_labels[np.argmax(emotion_probabilities)]

#         # Draw a rectangle around the detected face and label the emotion
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
#         # lanmarks=predictor(face)
#         # for point in range(68):
#         #     x=lanmarks.part(point).x
#         #     y=lanmarks.part(point).y
#         #     cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
#         cv2.putText(frame, predicted_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

#     # Display the frame
#     cv2.imshow("Emotion Detection", frame)

#     # Exit when the 'q' key is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the webcam and close all windows
# cap.release()
# cv2.destroyAllWindows()


In [ ]:
/

In [ ]:
/